# Synopsis

This notebook will explain the following topics and concepts:

- **Built in Statistical Functions** 

- **apply** and **applymap**

- **Calculating the return of an investment**
  - Normalized prices
  - Cumulative returns
  - Log returns

- **Correlation & Covariance**
  - Calculating
  - visualizing
  - rolling correlations

# Import packages

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 


# Create a Simple DataFrame

In [ ]:
serA = stats.randint(1,100).rvs(15)
serB = stats.randint(1000, 5000).rvs(15)

df = pd.DataFrame()
df['A'] = serA
df['B'] = serB

df.transpose()

# Built in Statistical Functions

The following functions can all be applied to a Series.

As a column is a Series, they can all be applied to a column or columns of a DataFrame or even an entire DataFrame

- Simple Functions
- Accumulators
- General Purpose Functions

## Simple Functions


- count() 
- min() 
- max() 
- sum() 
- mean()
- median() 
- std() 
- describe()

In [ ]:
df.count()
df.min()
df.max()
df.sum()
df.sum().sum()
df.mean()
df.median()
df.std()
df.describe()

## accumulators

- cumsum()
- cummin()
- cummax()
- cumprod()

In [ ]:
df.transpose()

In [ ]:
df.cumsum().transpose()

In [ ]:
df.cummin().transpose()

In [ ]:
df.cummax().transpose()

In [ ]:
df.cumprod().transpose()

## General purpose Functions

There are also a few general purpose functions

- diff()  - difference between adjacent values
- pct_change() - percentage change between adjacent values
- idxmin() - numerical index of minimum value in series (Series begin at index 0)
- idxmax() - numerical index of maximum value in series
- skew()
- kurt()
- quantile()

In [ ]:
df.pct_change().transpose()
df.diff().transpose()
df.idxmax()
df.idxmin()
df.skew()
df.kurt()
df.quantile(q=[0.25, 0.5, 0.75])

# Applying functions to Series and DataFrames

- You can easily apply arbitrary functions to DataFrames.

- Use the **apply()** function

- This method can be used to apply a function to a Series, Column, Columns or an entire DataFrame



## Create a simple DataFrame

In [ ]:
def makeDF():
    cols = ['A', 'B', 'C']
    rows = ['Row 1', 'Row 2', 'Row 3', 'Row 4']
    data = np.arange(12).reshape(4,3)

    return pd.DataFrame(data=data, columns=cols, index=rows)

df = makeDF()

df

## Apply across a row

Use the `sum()` method to sum each element of a **row**

In [ ]:
# Create a simple DataFrame
df = makeDF()

cols = ['A', 'B', 'C']
def custom_sum(row):
    return row.sum()

df['Row Result'] = df[cols].apply(custom_sum, axis=1)
df

## Apply along a column

In [ ]:
# Create a simple DataFrame
df = makeDF()

df.loc['Col Result'] = df.apply(custom_sum, axis=0)

df

## Apply
The default axis is 0

In [ ]:
# Create a simple DataFrame
df = makeDF()

def multiply_by_2(val):
    return val * 2

df['Row Result'] = df['C'].apply(multiply_by_2)

df

## Using a lambda

In [ ]:
# Create a simple DataFrame
df = makeDF()

# The lambda equivalent for the sum of each row of a DataFrame:
df['Row Result'] = df[cols].apply(lambda x:x.sum(), axis=1)
df

In [ ]:
# Create a simple DataFrame
df = makeDF()

# The lambda equivalent for the sum of each column of a DataFrame:
df.loc['Col Result']= df.apply(lambda x:x.sum(), axis=0)
df

In [ ]:
# The lambda equivalent for multiply by 2 on a Series
df['Row Result'] = df['C'].apply(lambda x:x*2)

df

## User Defined Functions

$y = max(0, cos(x))$

In [ ]:
df = makeDF()

def foo(x):
    y = np.cos(x) 
    return y if y > 0 else 0

df.map(foo)

## Functions that have parameters

In [ ]:
df = makeDF()

def bar(x, e):
    return x + (e**2)

df['A'].apply(func=bar,e=2)

df.apply(func=bar,e=2)

# Calculating the return of an investment


**Common Front Office Calculations**
- Normalized prices
- daily returns
- the log of returns

## Import some data from yahoo finance

### If yfinance is available

In [ ]:
import yfinance as yf

df_stocks =  yf.download(tickers='TSLA MSFT AAPL', start='2018-01-01')['Adj Close'].copy()

df_stocks.plot()

The problem with plotting the price is that its difficult to gauge how well the security has performed.

Better to first scale the prices to 1 and then plot.

A similar problem, when comparing 2 or more investments, also best to scale all prices to 1 and then plot, this makes for an easier comparison.

## Normalized Prices

The difference between price(t0) and price (t+1)

This is the same as cumulative daily returns

In [ ]:
df_stocks_Normed = df_stocks/df_stocks.iloc[0]

df_stocks_Normed.plot(grid=True)

The problem with normalized prices is that its static. 

Change the dates of the Dataframe and the original problem is still present.


In [ ]:
df_stocks_Normed['2021-Aug':'2023'].plot()

## Daily Returns

- When calculating the return of an investment or position, the product of daily returns can be used.

- This allows a direct comparison to be made between different instruments

- When backtesting technical analysis you will be employing this measure to compare a simple trading strategy against market performance.

- This is a very simple value to arrive at

- price / price(t-1)

- price(t-1) arrived at by using the time shift functions

- Use the `cumprod()` function to arrive at the payoff



Use `cumprod` to convert daily returns to stock performance

In [ ]:
df_stocks_daily_returns =  df_stocks / df_stocks.shift(1)

df_stocks_daily_returns.plot()

In [ ]:
df_stocks_daily_returns.cumprod().plot()

In [ ]:
df_stocks.head()

In [ ]:
df_stocks_daily_returns['2012':'2020'].cumprod().plot()

## Log of the Daily Returns

- Used more frequently than product of returns (above)

- When calculating the return of an investment or position, an accumulation of the log of daily returns is used.

- This allows a direct comparison to be made between different instruments

- When backtesting technical analysis you will be employing this measure to compare a simple trading strategy against market performance.

- This is a very simple value to arrive at

- `log (price / price(t-1))`

- Use a combination of `apply()` and  `cumsum()` function to arrive at the payoff


In [ ]:
df_stocks_log_returns = np.log(df_stocks / df_stocks.shift(1))

df_stocks_log_returns.plot()

In [ ]:
df_stocks_log_returns.cumsum().apply(np.exp).plot()

In [ ]:
df_stocks_log_returns['2012':'2020'].cumsum().apply(np.exp).plot()

# Correlation and Co-Variance

- Pandas has some convenient built-ins for calculating these.

- We'll Use some previous datasets for demonstration.

- Calculate the correlation and covariance between the daily percentage change of the Adjusted Close price of FANG Stocks and Gold Futures.

- Display the correlation

- Calculate the covariance of the same data

## Correlate Prices

### If yfinance is available

In [ ]:
df = yf.download(tickers='^GSPC JPM IBM F GLD BZ=F', start='2000-01-01')['Adj Close'].copy()


df.corr()

In [ ]:
# Note - Correlation and Covariance change over time
df.loc['2023'].corr()

## Correlate Daily Percentage Change

**Note** 

These correlations are different from the prices

In [ ]:
df_pct_change = df.pct_change(fill_method=None)


In [ ]:
df_pct_change.corr()

In [ ]:
df_pct_change.loc['2023'].corr()

## Calculate covariance

- Use the **cov()** function



In [ ]:
df_pct_change.cov()


# Covariance
df_pct_change.loc['2017'].cov()


## Use a scatter plot to display a visual of correlation


In [ ]:
from pandas.plotting import scatter_matrix
p = scatter_matrix(df_pct_change.loc['2017'], alpha=0.9, hist_kwds={'bins':50}, figsize=(18,6))

## Use a Heatmap

In [ ]:
import seaborn as sns

df_corr = df_pct_change.corr()

In [ ]:
sns.heatmap(df_corr)

In [ ]:
sns.heatmap(df_corr, annot=True)

### Advanced heatmap

In [ ]:
# Create a 2D mask / filter
mask = np.zeros(df_corr.shape, dtype=bool)
mask


In [ ]:
# Get triangle Upper Indices
mask[np.triu_indices(len(mask))] = True
mask

In [ ]:
# Mask out the upper part of the heat map
sns.heatmap(df_corr,annot=True,mask=mask)

## Rolling Correlations

In [ ]:
## Rolling Correlations

ax = df_pct_change['F'].rolling(window=252).corr(df_pct_change['IBM']).plot(figsize=(10, 6))  

# This line shows the corralation of both over the entire time period
# Note how the rolling correlation is much more telling

corr_full = df_pct_change[['F']].corrwith(df_pct_change['IBM']),

ax.axhline(corr_full, c='r');  